# Different similarity measurements 

https://www.youtube.com/watch?v=ZBCLIvznxnI&ab_channel=MachineLearningMindset

TODO: 
- check their correcting encoding to utf-8


other ressources:
- https://colab.research.google.com/github/janlukasschroeder/nlp-cheat-sheet-python/blob/master/NLP-Cheat-Sheet.ipynb
- 

what to compare: (always one to one comparison)
- job ads to cv
- ??jobs voc as basic voc because trained on that one
- sub categories of jobads to cv



good experiences:
- check how model does on: cv with words known from job vs cv with a lot of "new words"(words not in jobads)

new problems:
- how do i define the vocabulary for BERT model if resumes can have new words? real life example is trained on big dataset and thenuse cv to better learn?

In [2]:
#install all deppendencies
%pip install --upgrade pip
%pip install --upgrade protobuf


%pip install tensorflow
%pip install tensorflow-text

Note: you may need to restart the kernel to use updated packages.


                                              0.0/420.6 kB ? eta -:--:--
     --------------------------------       358.4/420.6 kB 7.4 MB/s eta 0:00:01
     -------------------------------------- 420.6/420.6 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.4 which is incompatible.
tensorflow-intel 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.12.0 requires tensorflow-estimator<2.13,>=2.12.0, but you have tensorflow-estimator 2.10.0 which is incompatible.


  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.7.0-py3-none-any.whl (2.4 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-

  Using cached tensorflow_text-2.10.0-cp310-cp310-win_amd64.whl (5.0 MB)
  Using cached tensorflow-2.10.1-cp310-cp310-win_amd64.whl (455.9 MB)
  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl (895 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-dat

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\Users\\Andru\\OneDrive\\Master Thesis\\BERT-XMLC-Skill-Extraction\\venv_310_64bit\\Lib\\site-packages\\tensorflow\\include\\external\\llvm-project\\mlir\\_virtual_includes\\BufferizationTransformOpsIncGen\\mlir\\Dialect\\Bufferization\\TransformOps\\BufferizationTransformOps.cpp.inc'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [3]:
import re
from collections import Counter
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text as tf_txt #pip install tensorflow-text

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\Andru\OneDrive\Master Thesis\BERT-XMLC-Skill-Extraction\venv_310_64bit\lib\site-packages\google\protobuf\internal\__init__.py)

In [1]:
text_init = "D_df dsdds"

text_plain = text_init.encode('UTF-16')
utf_text = tf.strings.unicode_transcode(text_plain, input_encoding='UTF-16', output_encoding='UTF-8')

print("utf_text:",utf_text)
utf_text_byte = utf_text.numpy()
returned_text_str = utf_text_byte.decode('UTF-8')
print("returned_text_str:",returned_text_str)


NameError: name 'tf' is not defined

## Shared vocabulary

- create universal vocabulary (union of individual vocs)
- create a shared vocabulary vector for each datatset:
   - for each word of universal vocabulary 1 if present 0 if not
- angular distance to measure the similarity

In [2]:
# https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch3/02_Bag_of_Words.ipynb?authuser=1#scrollTo=bMoADcrhJP2H

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

dataset = pd.read_csv("jobads_dataset.csv")
documents = dataset.text.values

#processed_docs = [doc.lower().replace(".","") for doc in documents]

processed_docs = documents


#look at the documents list
print("Our corpus: ", processed_docs[0:5])

count_vect = CountVectorizer()
#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

#see the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'man bites dog: ",bow_rep[1].toarray())

#Get the representation using this vocabulary, for a new text
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())

Our corpus:  ['providing support director admin manager assisting daily day day activities carry indoor sales network tele marketing exp willing learn also generate quotation invoicing etc sales coordination functions requirements preferably least 1 year relevant exp'
 "company overview google conventional company intend become one true share attributes world successful organizations focus innovation smart business practices comes mind even continue grow committed retaining small company feel google know every employee something important say every employee integral success provide individually tailored compensation packages comprised competitive salary bonus equity components along opportunity earn financial bonuses rewards googlers thrive small focused teams high energy environments believe ability technology change world passionate lives work information visit www google com careers area ux user experience forefront create intuitive innovative beautiful products people love strive l

In [3]:
# https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch3/02_Bag_of_Words.ipynb?authuser=1#scrollTo=bMoADcrhJP2H

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# no stemming and lemmatization done in jobads dataset
# example: "write, written, writing, writers" are present in dataset

#TODO: in resumes remove rare words

#------------load all
dataset_jobs = pd.read_csv("jobads_dataset_TEST.csv")
texts_jobs = dataset_jobs.text.values.tolist()
dataset_cv = pd.read_csv("resumes_dataset_TEST.csv")
texts_cv = dataset_cv.text.values.tolist()

print("texts_jobs:",type(texts_jobs),texts_jobs)
print("texts_cv:",type(texts_cv),texts_cv)

all_texts = texts_jobs + texts_cv

texts_jobs: <class 'list'> ['providing support director admin manager assisting daily day day activities carry indoor sales network tele marketing exp willing learn also generate quotation invoicing etc sales coordination functions requirements preferably least 1 year relevant exp', "company overview google conventional company intend become one true share attributes world successful organizations focus innovation smart business practices comes mind even continue grow committed retaining small company feel google know every employee something important say every employee integral success provide individually tailored compensation packages comprised competitive salary bonus equity components along opportunity earn financial bonuses rewards googlers thrive small focused teams high energy environments believe ability technology change world passionate lives work information visit www google com careers area ux user experience forefront create intuitive innovative beautiful products people

In [4]:



#----------------create universal vocabulary (union of individual vocs) & compare size to seperate vocs

def extract_voc(texts):
    count_vect_binary = CountVectorizer(binary=True) #only interested if the word is at least once in the dataset
    bow_rep = count_vect_binary.fit_transform(texts)
    voc = set(count_vect_binary.vocabulary_)

    print("VOCABULARY:")
    print("Our vocabulary: ", count_vect_binary.vocabulary_)
    print("voc of size:",len(voc))

    return voc, count_vect_binary

#Build a BOW representation for each dataset n union
universal_voc,count_vect_binary = extract_voc(all_texts)
jobs_voc,_ = extract_voc(texts_jobs)
cv_voc,_ = extract_voc(texts_cv)

print("universal_voc of size ",len(universal_voc),":",universal_voc)
print("jobs_voc of size ",len(jobs_voc),":",jobs_voc)
print("cv_voc of size ",len(cv_voc),":",cv_voc)



VOCABULARY:
Our vocabulary:  {'providing': 1034, 'support': 1267, 'director': 424, 'admin': 69, 'manager': 789, 'assisting': 143, 'daily': 360, 'day': 367, 'activities': 59, 'carry': 223, 'indoor': 660, 'sales': 1154, 'network': 868, 'tele': 1294, 'marketing': 798, 'exp': 506, 'willing': 1432, 'learn': 741, 'also': 89, 'generate': 578, 'quotation': 1052, 'invoicing': 697, 'etc': 490, 'coordination': 335, 'functions': 573, 'requirements': 1099, 'preferably': 987, 'least': 742, 'year': 1449, 'relevant': 1084, 'company': 281, 'overview': 927, 'google': 582, 'conventional': 329, 'intend': 684, 'become': 180, 'one': 898, 'true': 1343, 'share': 1189, 'attributes': 148, 'world': 1443, 'successful': 1259, 'organizations': 916, 'focus': 555, 'innovation': 670, 'smart': 1204, 'business': 208, 'practices': 985, 'comes': 270, 'mind': 823, 'even': 494, 'continue': 320, 'grow': 593, 'committed': 274, 'retaining': 1127, 'small': 1203, 'feel': 537, 'know': 722, 'every': 497, 'employee': 456, 'somethin

In [5]:
#-----------------create a shared vocabulary vector for each datatset (for each word of universal vocabulary 1 if present 0 if not)
#Get the representation using this vocabulary, for a new text

#EXAMPLE
temp = count_vect_binary.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())



import numpy as np

def get_shared_vocabulary_vector(texts,count_vect_binary):
    voc_presence = count_vect_binary.transform(texts)
    vector = np.array(voc_presence.toarray())
    total = np.sum(vector, axis=0)
    binary = total.astype(bool).astype(int)
    return binary



job_shared_vocabulary_vector = get_shared_vocabulary_vector(texts_jobs,count_vect_binary)
print("get_shared_vocabulary_vector(texts_job) with total ",sum(job_shared_vocabulary_vector),":",job_shared_vocabulary_vector)

cv_shared_vocabulary_vector = get_shared_vocabulary_vector(texts_cv,count_vect_binary)
print("get_shared_vocabulary_vector(texts_cv) with total ",sum(cv_shared_vocabulary_vector),":",cv_shared_vocabulary_vector)




Bow representation for 'dog and dog are friends': [[0 0 0 ... 0 0 0]]
get_shared_vocabulary_vector(texts_job) with total  719 : [0 1 0 ... 1 0 0]
get_shared_vocabulary_vector(texts_cv) with total  970 : [1 0 1 ... 1 1 1]


In [6]:
#-----------------angular difference between shared voc vectors
# FROM PAPER (domain similarities):
# The angular distance between two of these individual dataset vectors should signify their similarity.
# This should give an indication of whether similar language is used between the datasets.

# https://en.wikipedia.org/wiki/Angular_distance

# https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python/13849249#13849249

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


# https://math.stackexchange.com/questions/2874940/cosine-similarity-vs-angular-distance
# sim(u.v) = (1 - arccos(cosine_similarity(u, v)) / pi)
#   with:
# cosine_similarity(u, v) = (u . v) / (||u|| . ||v||)

#TODO unedrstand diff between these 2 formulas: 
#      I think: BUT REALLY NOT SURE
#           - 1st is angle between the two vectors:
#               this does not take into account the magnitudes (angular_dist(u,v)=angular_dist(||u||,||v||)
#           - 2nd is cos of angle between the two
#               this takes into account the magnitudes (cos(u,v)=/=cos(||u||,||v||)


angle_between_job_cv = angle_between(job_shared_vocabulary_vector,cv_shared_vocabulary_vector)
print("angle_between_job_cv:",angle_between_job_cv)

size_universal_voc = len(universal_voc)
vector_universal_voc = np.ones(size_universal_voc)
angle_between_universal_job = angle_between(job_shared_vocabulary_vector,vector_universal_voc)
print("angle_between_universal_job:",angle_between_universal_job)
angle_between_universal_cv = angle_between(cv_shared_vocabulary_vector,vector_universal_voc)
print("angle_between_universal_cv:",angle_between_universal_cv)


angle_between_job_cv: 1.2842999106252504
angle_between_universal_job: 0.7905599894290355
angle_between_universal_cv: 0.6145060669325315


## TD-IDF

In [7]:
# https://colab.research.google.com/github/janlukasschroeder/nlp-cheat-sheet-python/blob/master/NLP-Cheat-Sheet.ipynb#scrollTo=_s-zKz_z2miy

from sklearn.feature_extraction.text import TfidfVectorizer

document_corpus = [
  "Dog bites man", 
  "Man bites dog", 
  "Dog eats meat", 
  "Man eats food"
]

tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(document_corpus)

print("IDF for all words in the vocabulary")
print(tfidf.idf_)
print("\nAll words in the vocabulary.")
print(tfidf.get_feature_names_out())

temp = tfidf.transform(["Dog bites man"])

print("\nTF-IDF representation for 'Dog bites man':\n", temp.toarray())

IDF for all words in the vocabulary
[1.51082562 1.22314355 1.51082562 1.91629073 1.22314355 1.91629073]

All words in the vocabulary.
['bites' 'dog' 'eats' 'food' 'man' 'meat']

TF-IDF representation for 'Dog bites man':
 [[0.65782931 0.53256952 0.         0.         0.53256952 0.        ]]


## Cosine Similarity (&distance)